In [149]:
import os
import pandas as pd
import folium
import geojson
from geojson import FeatureCollection, Feature
from branca.colormap import linear
import glob

In [150]:
# Defining the CSV files path
DATA_DIR = "resources/data"

In [151]:
# Importing the Vodafone DataSet from CSV files

dfVodafone = []

for csv_file in glob.glob(os.path.join(DATA_DIR, "*.csv")):
    dfVodafone.append(pd.read_csv(
        csv_file,
        usecols=['Grid_ID', 'Datetime', 'C4'],
        dtype={'Grid_ID': 'int64', 'C4': 'float64'},
        parse_dates=['Datetime'],
        date_format="%Y-%m-%dT%H:%M:%S.%fZ"
    ))

dfVodafone = pd.concat(dfVodafone, axis=0)

In [152]:
# Displaying the information about the dataframe
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287678405 entries, 0 to 33402531
Data columns (total 3 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Grid_ID   int64         
 1   Datetime  datetime64[ns]
 2   C4        float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 8.6 GB


In [153]:
# Displaying simple descriptive metrics
dfVodafone.describe().transpose().apply(lambda s: s.apply('{0:.0f}'.format))

,count,mean,min,25%,50%,75%,max,std
Grid_ID,287678405,1881,1,936,1873,2831,3743,1085
Datetime,287678405,.0f,.0f,.0f,.0f,.0f,.0f,nan
C4,287678405,16,0,0,3,12,8789,46


In [154]:
# Filtering by night time
dfVodafone = dfVodafone.set_index('Datetime').between_time('02:00', '07:00').reset_index()

In [155]:
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60767031 entries, 0 to 60767030
Data columns (total 3 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Datetime  datetime64[ns]
 1   Grid_ID   int64         
 2   C4        float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.4 GB


In [156]:
# Importing the Vodafone Metadata with information about the cells
dfGrids = pd.read_csv("resources/metadata/wktComplete.csv", encoding="ISO-8859-1")

In [157]:
dfGrids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   grelha_id    3743 non-null   int64  
 1   dicofre      3743 non-null   int64  
 2   entity_id    3743 non-null   object 
 3   entity_type  3743 non-null   object 
 4   freguesia    3743 non-null   object 
 5   freguesias   3743 non-null   object 
 6   grelha_x     3743 non-null   int64  
 7   grelha_y     3743 non-null   int64  
 8   latitude     3743 non-null   float64
 9   longitude    3743 non-null   float64
 10  nome         3743 non-null   object 
 11  objectid     3743 non-null   int64  
 12  position     3743 non-null   object 
 13  wkt          3743 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 409.5+ KB


In [158]:
# Removing columns that we don't care about
dfGrids = dfGrids[["grelha_id", "freguesia", "latitude", "longitude", "nome", "position"]]

In [159]:
dfGrids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   grelha_id  3743 non-null   int64  
 1   freguesia  3743 non-null   object 
 2   latitude   3743 non-null   float64
 3   longitude  3743 non-null   float64
 4   nome       3743 non-null   object 
 5   position   3743 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 175.6+ KB


In [160]:
# Grouping by Grid ID and Day, aggregating by the minimum number of terminals each night for every cell
dfSleepByDay = dfVodafone.assign(Date=dfVodafone["Datetime"].dt.date).groupby(["Grid_ID", "Date"], as_index=False)["C4"].min()
dfSleepByDay["Date"] = pd.to_datetime(dfSleepByDay["Date"])
dfSleepByDay.rename(columns={"C4": "Sleeping"}, inplace=True)

In [161]:
dfSleepByDay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1301757 entries, 0 to 1301756
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype         
---  ------    --------------    -----         
 0   Grid_ID   1301757 non-null  int64         
 1   Date      1301757 non-null  datetime64[ns]
 2   Sleeping  1301757 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 29.8 MB


In [162]:
# Grouping by Grid ID, aggregating by the mean number of minimum of terminals each night
dfSleepMean = dfSleepByDay.groupby(["Grid_ID"], as_index=False)["Sleeping"].mean()

In [163]:
dfSleepMean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Grid_ID   3743 non-null   int64  
 1   Sleeping  3743 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 58.6 KB


In [164]:
# Adding the information about the cells to our dataframe
dfSleepMean = dfSleepMean.merge(dfGrids, left_on='Grid_ID', right_on='grelha_id')

In [165]:
dfSleepMean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Grid_ID    3743 non-null   int64  
 1   Sleeping   3743 non-null   float64
 2   grelha_id  3743 non-null   int64  
 3   freguesia  3743 non-null   object 
 4   latitude   3743 non-null   float64
 5   longitude  3743 non-null   float64
 6   nome       3743 non-null   object 
 7   position   3743 non-null   object 
dtypes: float64(3), int64(2), object(3)
memory usage: 234.1+ KB


In [166]:
# Creating a linear color map
colormap = linear.YlOrRd_09.scale(
    dfSleepMean["Sleeping"].min(), dfSleepMean["Sleeping"].max()
)

colormap

In [167]:
Sleeping_dict = dfSleepMean.set_index("Grid_ID")["Sleeping"]
color_dict = {key: colormap(Sleeping_dict[key]) for key in Sleeping_dict.keys()}

In [168]:
m = folium.Map(location=[38.743094, -9.145999], zoom_start=13)

heatmap_features = []
for index, sleepByDay in dfSleepMean.iterrows():
    feature = Feature(
        geometry=geojson.loads(sleepByDay["position"].replace("'", '"'))[0],
        properties={
            "Grid_ID": sleepByDay["Grid_ID"],
            "name": sleepByDay["nome"],
            "freguesy": sleepByDay["freguesia"],
            "sleeping": sleepByDay["Sleeping"],
        }
    )
    heatmap_features.append(feature)

feature_collection = FeatureCollection(heatmap_features)

tooltip = folium.GeoJsonTooltip(
    fields=["name", "freguesy", "sleeping"],
    aliases=["Zona:", "Freguesia:", "A Dormir:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    data=feature_collection,
    style_function=lambda feature: {
        "fillColor": color_dict[feature.properties["Grid_ID"]],
        "color": "black",
        "weight": 0,
        "fillOpacity": .5,
    },
    tooltip=tooltip
).add_to(m)

colormap.add_to(m)

m.save('dist/sleeping_cells_heatmap.html')

m 

In [186]:
# Showing top 20 cells
dfTop = dfSleepMean.nlargest(20, "Sleeping")[["Grid_ID", "nome", "freguesia", "Sleeping"]]
dfTop.to_csv("dist/dfTop.csv", decimal=',')
dfTop

,Grid_ID,nome,freguesia,Sleeping
686,687,Praça da Figueira,Santa Maria Maior,462.986475
571,572,Baixa,Santa Maria Maior,460.420173
511,512,Camões,Misericórdia,440.016533
624,625,Bairro Alto,Misericórdia,390.726504
628,629,Baixa,Santa Maria Maior,356.332350
627,628,Rossio,Santa Maria Maior,321.364441
516,517,Baixa,Santa Maria Maior,311.369884
682,683,Bairro Alto,Misericórdia,294.320745
510,511,Bica,Misericórdia,291.726648
570,571,Baixa,Santa Maria Maior,285.296448


In [187]:
# Group by freguesy, aggregate by sum
dfSleepMeanFreg = dfSleepMean.groupby("freguesia")["Sleeping"].sum().reset_index()

In [188]:
dfSleepMeanFreg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   freguesia  24 non-null     object 
 1   Sleeping   24 non-null     float64
dtypes: float64(1), object(1)
memory usage: 512.0+ bytes


In [189]:
# Export and show the avg number of tourists sleeping in each of the counties (freguesias)
dfTopFreg = dfSleepMeanFreg.sort_values(by='Sleeping', ascending=False)
dfTopFreg.to_csv("dist/dfTopFreg.csv", decimal=',')
dfTopFreg

,freguesia,Sleeping
20,Santa Maria Maior,6657.294727
21,Santo António,4884.286473
4,Arroios,4223.404954
15,Misericórdia,3750.041776
5,Avenidas Novas,3545.483713
12,Estrela,1311.822865
23,São Vicente,1280.318166
17,Parque das Nações,1082.158388
18,Penha de França,884.560295
9,Campo de Ourique,661.539370


In [190]:
# Summing the number of minimum of terminals each day, and getting the mean for each Weekday
dfTotalSleepByDay = dfSleepByDay.groupby("Date", as_index=False)["Sleeping"].sum()

dfTotalSleepByWeekday = dfTotalSleepByDay.assign(Weekday=dfSleepByDay['Date'].dt.weekday)
dfTotalSleepByWeekday = dfTotalSleepByWeekday.assign(WeekdayName=dfSleepByDay['Date'].dt.day_name())
dfTotalSleepByWeekdayMean = dfTotalSleepByWeekday.groupby(["Weekday", "WeekdayName"], as_index=False)["Sleeping"].mean()

In [191]:
dfTotalSleepByWeekdayMean.to_csv("dist/dfSleepByWeekdayMean.csv", decimal=',')
dfTotalSleepByWeekdayMean

,Weekday,WeekdayName,Sleeping
0,0,Monday,30364.113608
1,1,Tuesday,31486.510787
2,2,Wednesday,31561.427942
3,3,Thursday,30882.504328
4,4,Friday,32375.068839
5,5,Saturday,34586.323268
6,6,Sunday,34234.507693


In [192]:
# Getting the mean number of tourists sleeping in all Lisbon for each Month
dfTotalSleepByMonth = dfTotalSleepByDay.assign(Month=dfTotalSleepByDay['Date'].dt.month)
dfTotalSleepByMonth = dfTotalSleepByMonth.assign(MonthName=dfTotalSleepByDay['Date'].dt.month_name())
dfTotalSleepByMonthMean = dfTotalSleepByMonth.groupby(["Month", "MonthName"], as_index=False)["Sleeping"].mean()

In [193]:
dfTotalSleepByMonthMean.to_csv("dist/dfSleepByMonthMean.csv", decimal=',')
dfTotalSleepByMonthMean

,Month,MonthName,Sleeping
0,1,January,32780.464617
1,2,February,36117.950726
2,3,March,41403.539330
3,4,April,44575.989296
4,5,May,45183.134832
5,6,June,46713.660335
6,7,July,7451.171935
7,8,August,7245.533226
8,9,September,23250.798658
9,10,October,41305.379986
